In [1]:
import pandas as pd
import numpy as np
import scipy
import statsmodels.stats.weightstats

## 2.2 潜在的結果変数の枠組み：理論

### 2.3 例示用データ

#### ファイル読み込み

In [2]:
data02 = pd.read_csv('./causality/data02.csv')
display(data02)

,x1,y3,t1,y0,y1,y0t,y1t
0,74,76,1,NaN,76.0,68,76
1,82,75,0,75.0,NaN,75,84
2,72,75,1,NaN,75.0,65,75
3,96,84,0,84.0,NaN,84,97
4,83,75,0,75.0,NaN,75,84
5,72,74,1,NaN,74.0,65,74
6,85,76,0,76.0,NaN,76,87
7,87,77,0,77.0,NaN,77,89
8,86,77,0,77.0,NaN,77,87
9,77,80,1,NaN,80.0,70,80


#### 要約統計量表示

In [3]:
display(data02.describe())

,x1,y3,t1,y0,y1,y0t,y1t
count,20.000000,20.000000,20.000000,14.000000,6.000000,20.000000,20.000000
mean,81.950000,76.600000,0.300000,77.785714,73.833333,73.800000,83.850000
std,8.999854,5.245549,0.470162,4.281958,6.615638,7.898034,8.430989
min,58.000000,61.000000,0.000000,72.000000,61.000000,52.000000,61.000000
25%,76.250000,75.000000,0.000000,75.000000,74.250000,69.500000,79.250000
50%,83.500000,76.500000,0.000000,77.000000,75.500000,75.000000,84.500000
75%,87.250000,80.000000,1.000000,80.000000,76.750000,78.500000,89.000000
max,96.000000,87.000000,1.000000,87.000000,80.000000,87.000000,97.000000


## 2.3 処置効果１：

### 表2.4 個体因果効果

#### 教科書通りのやり方 (Jupyterだと読みづらい）

In [4]:
data02.y1t - data02.y0t

0      8
1      9
2     10
3     13
4      9
5      9
6     11
7     12
8     10
9     10
10     9
11    10
12    10
13     9
14    12
15    12
16    10
17     9
18    10
19     9
dtype: int64

#### 人間に読みやすい方法

In [5]:
pd.DataFrame({
    'y1t- y0t': data02.y1t - data02.y0t,
    'y3 - x1': data02.y3 - data02.x1,
    'y1 - y0': data02.y1 - data02.y0,
})

,y1t- y0t,y3 - x1,y1 - y0
0,8,2,NaN
1,9,-7,NaN
2,10,3,NaN
3,13,-12,NaN
4,9,-8,NaN
5,9,2,NaN
6,11,-9,NaN
7,12,-10,NaN
8,10,-9,NaN
9,10,3,NaN


## 2.4 処置効果2: 平均処置効果

 ###　表2.5 平均処置効果(ATE)

In [6]:
np.mean(data02.y1t) - np.mean(data02.y0t)

10.049999999999997

#### Series.mean()を利用

In [7]:
display(data02.y1t.mean() - data02.y0t.mean())

10.049999999999997

In [8]:
np.mean(data02.y3) - np.mean(data02.x1)

-5.3500000000000085

#### NaN除外した平均はnp.nanmeanを利用

In [9]:
m1 = np.nanmean(data02.y1)
m0 = np.nanmean(data02.y0)
m1 - m0

-3.952380952380963

#### Series.mean(skipna=True)でも可能

In [10]:
_m1 = data02.y1.mean(skipna=True)
_m0 = data02.y0.mean(skipna=True)
_m1 - _m0

-3.952380952380963

## 2.5 処置効果3: 処置群の平均処置効果

### 表2.6 真のATT

In [11]:
mt1 = data02[data02.t1 == 1].y1t.mean()
mt0 = data02[data02.t1 == 1].y0t.mean()
mt1 - mt0

9.333333333333329

#### DataFrame.queryを利用

In [12]:
_mt1 = data02.query('t1 == 1').y1t.mean()
_mt0 = data02.query('t1 == 1').y0t.mean()
_mt1 - _mt0

9.333333333333329

## 2.6 無作為割付けによる分析の例

### 表2.7 無作為割付による分析

#### np.random.uniform: 一様分布を生成

In [13]:
np.random.seed(1)
r0 = np.random.uniform(0,1, size=20)
display(r0)
r1 = np.round(r0, 0)
display(r1)

array([4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
       1.46755891e-01, 9.23385948e-02, 1.86260211e-01, 3.45560727e-01,
       3.96767474e-01, 5.38816734e-01, 4.19194514e-01, 6.85219500e-01,
       2.04452250e-01, 8.78117436e-01, 2.73875932e-02, 6.70467510e-01,
       4.17304802e-01, 5.58689828e-01, 1.40386939e-01, 1.98101489e-01])

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 0.])

#### 教科書の乱数振り分け結果

In [14]:
_r1 = np.array([0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1])

In [15]:
# 教科書の結果に合わせる
r1 = _r1
y2 = np.empty(shape=(20))
y2.fill(np.nan)
y2[r1 == 1] = data02.y1t[r1 == 1]
y2[r1 == 0] = data02.y0t[r1 == 0]
display(y2)

array([68., 75., 75., 97., 75., 74., 87., 89., 87., 70., 87., 75., 77.,
       52., 93., 72., 82., 89., 80., 87.])

In [16]:
mr1 = np.mean(y2[r1==1])
mr2 = np.mean(y2[r1==0])
display(mr1)
display(mr2)
display(mr1 - mr2)

85.18181818181819

72.66666666666667

12.515151515151516

#### DataFrameを活用

In [17]:
np.random.seed(1)
df = data02.copy()

df['r0'] = np.random.uniform(0,1, size=20)
df['r1'] = df['r0'].apply(np.round, 0)
# 教科書の結果に合わせる
df['_r1'] = _r1
df['y2'] = df.apply(lambda x: x['y1t'] if x['_r1'] == 1 else x['y0t'], axis = 1)
display(df)

,x1,y3,t1,y0,y1,y0t,y1t,r0,r1,_r1,y2
0,74,76,1,NaN,76.0,68,76,0.417022,0.0,0,68
1,82,75,0,75.0,NaN,75,84,0.720324,1.0,0,75
2,72,75,1,NaN,75.0,65,75,0.000114,0.0,1,75
3,96,84,0,84.0,NaN,84,97,0.302333,0.0,1,97
4,83,75,0,75.0,NaN,75,84,0.146756,0.0,0,75
5,72,74,1,NaN,74.0,65,74,0.092339,0.0,1,74
6,85,76,0,76.0,NaN,76,87,0.186260,0.0,1,87
7,87,77,0,77.0,NaN,77,89,0.345561,0.0,1,89
8,86,77,0,77.0,NaN,77,87,0.396767,0.0,1,87
9,77,80,1,NaN,80.0,70,80,0.538817,1.0,0,70


In [18]:
_mr1 = df.query('_r1 == 1')['y1t'].mean()
_mr0 = df.query('_r1 == 0')['y0t'].mean()

pd.DataFrame([{
    'mr1': _mr1,
    'mr0': _mr0,
    'mr1 - mr0': _mr1 - _mr0,
}], index=['result']).T

,result
mr1,85.181818
mr0,72.666667
mr1 - mr0,12.515152


## 2.10 2標本t検定

#### scipy版

In [19]:
ttest_result = scipy.stats.ttest_ind(df.query('_r1 == 1').y2, df.query('_r1 == 0').y2, equal_var=False)
display(ttest_result)

Ttest_indResult(statistic=3.2178051903997393, pvalue=0.005800978379825229)

#### statsmodels版（nassyemonはこっちを使う）

In [20]:
cm = statsmodels.stats.weightstats.CompareMeans.from_data(df.query('_r1 == 1').y2, df.query('_r1 == 0').y2)
_t_test_result = cm.ttest_ind(alternative='two-sided', usevar='unequal')
_ttest_confint = cm.tconfint_diff(alternative='two-sided', usevar='unequal')
print(_t_test_result)
print(_ttest_confint)

pd.DataFrame(
    [*_t_test_result, *_ttest_confint],
    index=['t-statistic', 'p-value', 'df', 'confint_low', 'confint_high'],
    columns=['welch-test result'],
)

(3.2178051903997393, 0.005800978379825222, 14.877550058900113)
(4.219264246524833, 20.811038783778198)


,welch-test result
t-statistic,3.217805
p-value,0.005801
df,14.877550
confint_low,4.219264
confint_high,20.811039
